In [1]:
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension
#conda install -c conda-forge gmaps
#!jupyter nbextension enable --py --sys-prefix gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
#loadcsv

file_df=pd.read_csv("../"+ output_data_file)
file_df



,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#configure gmaps

gmaps.configure(api_key=g_key)

#store lat and lng

locations = file_df[["Lat","Lng"]]
humidity =file_df["Humidity"]

In [5]:
#heatmap
heatmap = gmaps.figure(center = [0,0] ,zoom_level = 2)

#add layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
heatmap.add_layer(heat_layer)

heatmap



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#weather criteria
wc=file_df.loc[(file_df["Wind Speed"] <= 10) & (file_df["Cloudiness"] == 0) & \
                                   (file_df["Max Temp"] >= 70) & (file_df["Max Temp"] <= 80)].dropna()
wc

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = wc.reset_index(drop=True)
hotel_df["Hotel"] =""

#set paramaters

params = {"radius":5000,"type":"hotels", "keyword":"hotels", "key":g_key}

#loop
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    cityname = row["City"]
    
    params["location"] = f'{lat},{lng}'
    
    baseurl="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel = requests.get(baseurl, params=params).json()
    
    
    print(json.dumps(hotel, indent=4, sort_keys=True))
    
    results = hotel['results']
    
    #just in case?
    try:
        hotel_df.loc[index, "Hotel Name"]=hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Ignore Result.....")
        
       


print("-------End of Search-------")
    











{
    "html_attributions": [],
    "next_page_token": "ATtYBwJclSqGWtwtTt5BYP8DsOvpQEOThIeDlqLnPcEFCWBPIMF4aR7UwSjoGpGrzxZ7gEWJRuLhZJtzO_eWNBIBmDzpg_gp_eKoR_g-27Ce98oXUs1kI6hLvRcH1cpyDao7dZPCfn5FzpWc_F6x8EKb0jcKMDAxTpI861IsNzMQJL1EXWmVFdRN69fCdfABT64rqMFw2HPHfC1zxO_lBXN1FtkwgL0EEVb_xnQXnmwqBPLgStTsiRKLRpPD-RKx8rJxKFucSl3M5qSAE0paz7WgFgda5zJ0P65d3-oCERHx56fybZKrZuLqP_GyS11y-p8XINRyvcYb4qJeCjO8ywNfDEfnP7asDrhWmm8FI3CThNzspDtuuU1bRr-ZYHcMrbu9WLZcPL9guAgmbyUR2gmqbO8vGxxlGFjQEbzgSomnaeWNFR2bTKYGhatwPLSML2LnDA",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.6425569,
                    "lng": -46.0168
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.64122037010728,
                        "lng": -46.01548067010727
                    },
                    "southwest": {
                        "lat": -20.64392

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -30.1778807,
                    "lng": -50.2035006
                },
                "viewport": {
                    "northeast": {
                        "lat": -30.17650502010729,
                        "lng": -50.20220167010728
                    },
                    "southwest": {
                        "lat": -30.17920467989273,
                        "lng": -50.20490132989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel Castelo",
            "photos": [
                {
                    "height": 1800,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/108481048178480058140\">Nanda</a>"

{
    "html_attributions": [],
    "next_page_token": "ATtYBwL6OAEnxV8e0c3GVWUOjOYMKRfdt2DltA47qN8vYEXHjqK_0YPAtDQvPSFupOWm2YY-bFyBvybhHs8sSb4m4-QzccZQpkbCwK-_sylS4ZKVloR0PboM7fzNxYm70XH_VVr9RcmVUY-lNVJbLOtxP8KF-CQ11lm4qUar2mgKIpHVv7I08W30qhRVgXTEGhJclTwA-RMJcPCOWIJNWPXoDJXngswdGE77UKXyL9gOeyRyMm9wFil1QevboFK2q_r7Nsq0LGIH-Mw4ZZnhKeynuBatMcOhZjcJ1IEyxqqnxoQyxfVC4emboS7imz8d0E60Ev48Wy3NEJ4B3KFZ259tuy_Tlh5aVtNyZKLT_g8--Vi98vEQgaWzKNaqpcwdhvgMdNXcmdiK6yrAxr4qxKPX53m0w3uOChnj-0wVv3-AmFKKOYFHizgZ_Yg7Eb4Gw0UVnQ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.3000259,
                    "lng": 44.2679367
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.29871172010727,
                        "lng": 44.26928077989272
                    },
                    "southwest": {
                        "lat": -20.3014

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Ignore Result.....
{
    "html_attributions": [],
    "next_page_token": "ATtYBwLj1o03xnnfxcARQjBWO2DbhMUjIrPIJX3Ql4rqLfPMooHYG8xERCNxWkG8_kHzErMNc0_QJlZ0FrXEBo_xiNEravkeDAX9I2kybdseq4b0KjQcaMKNgeU_Mw7OuhTuL6D7Q0sQI-Dg1pG4vwobOkVG_09K-qqhFt0Pd9m21ivnQLrsJ6h3lSCLIaUrbPbSfXDQT_alzrkZoK7-4_6SPOaK90LnbspyzzvGTMG1x16UuFCD9Oopt2WDnjiiWKD8pD8Sqrln66cQkzvt68yvqYLIt9jt2D4oWWiqZwan5HMVKCDr7tKbXSjUWSVgmzWjoOzR90QKYo_oHDUqegK1zx3g3HEMGjuz05OE1tecmQFMA4o52ezpjgC5wyG7E959wQJbUVjiqmjRegFJzdA5M5QIE-BjeVhddfIZ4W3R7wL__gMgEkAAUQJxs1WFl-_gFg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 35.2067501,
                    "lng": -2.9121019
                },
                "viewport": {
                    "northeast": {
                        "lat": 35.20815157989272,
                        "lng": -2.91097077010727

In [17]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel,Hotel Name
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,,Suites de Luxo em Escarpas do Lago
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,,Hotel Castelo
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,,Palissandre Cote Ouest resort & SPA
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,,NaN
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,,"Hotel Marchica Lagoon Resort, Nador Morocco"
5,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,,Mount Mogok Hotel
6,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,,Ghaem Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

NameError: name 'fig' is not defined